In [1]:
import numpy as np
import sklearn
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from matplotlib import rc
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
from matplotlib.ticker import MaxNLocator
import scipy.signal
from xgboost.sklearn import XGBClassifier

/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
df = pd.read_csv('../csvs/initial_orbital_elements.csv', index_col=0)
#del df['instability_time']
#del df['Rel_Eerr_short']
#Nrows = df.shape[0]
#print Nrows, len(np.where(df['Stable']==0)[0])
print(df.shape)
df.head()

(25000, 32)


,runstring,Stable,instability_time,Rel_Eerr,RHill12,RHill23,beta12,beta23,m1,a1,...,Omega2,f2,m3,a3,P3,e3,pomega3,inc3,Omega3,f3
0,0000000.bin,0,1.235644e+07,1.099985e-04,0.026033,0.023222,8.508586,15.826967,3.414016e-05,1.0,...,1.573978,-0.567081,1.826900e-06,1.589043,2.003050,0.002515,-0.108162,0.065498,-1.192440,-0.614709
1,0000001.bin,0,1.348749e+05,1.388587e-08,0.022704,0.031284,8.397002,15.367618,2.529838e-07,1.0,...,-1.749616,1.437355,1.955797e-05,1.671403,2.160777,0.072450,-0.343874,0.046939,-0.619350,0.192203
2,0000002.bin,0,1.276381e+04,3.048261e-04,0.036304,0.032149,3.461288,25.229467,7.380789e-05,1.0,...,-0.032550,-0.317958,1.477931e-07,1.936762,2.695151,0.003395,3.419165,0.014535,2.795101,1.685406
3,0000003.bin,1,1.000000e+09,6.492574e-10,0.011704,0.014963,18.513681,19.145889,5.174804e-07,1.0,...,0.946351,-2.294238,1.287851e-06,1.503164,1.842927,0.057380,-0.829096,0.098014,-2.995941,-1.625763
4,0000004.bin,0,8.791888e+01,9.336734e-07,0.006199,0.008634,5.494196,2.928938,5.106746e-07,1.0,...,-0.455684,-1.074927,1.542321e-06,1.059346,1.090326,0.010325,-1.214839,0.011840,-1.253823,1.215392


In [5]:
df["sumRH"] = df["beta12"] + df["beta23"]

In [8]:
from sklearn import metrics
Nvalues = 100
precision, recall, thresholds = np.zeros(Nvalues), np.zeros(Nvalues), np.zeros(Nvalues)
for i, thresh in enumerate(np.linspace(10.,70.,Nvalues)):
    pred = df["sumRH"] > thresh
    thresholds[i] = thresh
    precision[i] = metrics.precision_score(df['Stable'].values, pred)
    recall[i] = metrics.recall_score(df['Stable'].values, pred)
    print("thresh = {0:.6}, acc = {1:.6}, prec = {2:.6}, rec = {3:.6}".format(thresholds[i], metrics.accuracy_score(df['Stable'].values, pred), precision[i], recall[i]))

thresh = 10.0, acc = 0.43336, prec = 0.411564, rec = 1.0
thresh = 10.6061, acc = 0.43908, prec = 0.414024, rec = 1.0
thresh = 11.2121, acc = 0.4458, prec = 0.416944, rec = 0.999899
thresh = 11.8182, acc = 0.45316, prec = 0.420191, rec = 0.999798
thresh = 12.4242, acc = 0.46012, prec = 0.423315, rec = 0.999798
thresh = 13.0303, acc = 0.46864, prec = 0.427191, rec = 0.999596
thresh = 13.6364, acc = 0.4762, prec = 0.430684, rec = 0.999294
thresh = 14.2424, acc = 0.48456, prec = 0.434613, rec = 0.99889
thresh = 14.8485, acc = 0.49268, prec = 0.438495, rec = 0.998385
thresh = 15.4545, acc = 0.49976, prec = 0.441916, rec = 0.997477
thresh = 16.0606, acc = 0.50972, prec = 0.446853, rec = 0.996669
thresh = 16.6667, acc = 0.51888, prec = 0.451461, rec = 0.995055
thresh = 17.2727, acc = 0.52892, prec = 0.456666, rec = 0.993944
thresh = 17.8788, acc = 0.53836, prec = 0.461643, rec = 0.991825
thresh = 18.4848, acc = 0.5492, prec = 0.467531, rec = 0.989705
thresh = 19.0909, acc = 0.56036, prec = 0.

/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
